# 1. Download Data and wordvec

In [1]:
!mkdir data
!wget -P data https://nlp.stanford.edu/data/coqa/coqa-train-v1.0.json
!wget -P data https://nlp.stanford.edu/data/coqa/coqa-dev-v1.0.json
!mkdir wordvecs
!wget -P wordvecs http://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip
!unzip -d wordvecs wordvecs/glove.42B.300d.zip
!wget -P wordvecs http://nlp.stanford.edu/data/wordvecs/glove.840B.300d.zip
!unzip -d wordvecs wordvecs/glove.840B.300d.zip

mkdir: cannot create directory ‘data’: File exists
--2020-08-20 05:06:46--  https://nlp.stanford.edu/data/coqa/coqa-train-v1.0.json
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json [following]
--2020-08-20 05:06:47--  http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49001836 (47M) [application/json]
Saving to: ‘data/coqa-train-v1.0.json’

coqa-train-v1.0.jso 100%[===================>]  46.73M  13.7MB/s    in 3.9s    

2020-08-20 05:06:51 (11.9 MB/s) - ‘data/coqa-train-v1.0.json’ saved [49001836/49001836]


# 2. Activate CoreNLP server
Execute this command on the terminal in this directory
``` $ java -mx4g -cp lib/stanford-corenlp-3.9.1.jar edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000```

# 3. Preprocess Data

In [23]:
import json
import re
import time
import string
from pycorenlp import StanfordCoreNLP

nlp = StanfordCoreNLP('http://localhost:9000')
UNK = 'unknown'


def _str(s):
    """ Convert PTB tokens to normal tokens """
    if (s.lower() == '-lrb-'):
        s = '('
    elif (s.lower() == '-rrb-'):
        s = ')'
    elif (s.lower() == '-lsb-'):
        s = '['
    elif (s.lower() == '-rsb-'):
        s = ']'
    elif (s.lower() == '-lcb-'):
        s = '{'
    elif (s.lower() == '-rcb-'):
        s = '}'
    return s


def process(text):
    paragraph = nlp.annotate(text, properties={
                             'annotators': 'tokenize, ssplit',
                             'outputFormat': 'json',
                             'ssplit.newlineIsSentenceBreak': 'two'})

    output = {'word': [],
              # 'lemma': [],
              # 'pos': [],
              # 'ner': [],
              'offsets': []}

    for sent in paragraph['sentences']:
        for token in sent['tokens']:
            output['word'].append(_str(token['word']))
            # output['lemma'].append(_str(token['lemma']))
            # output['pos'].append(token['pos'])
            # output['ner'].append(token['ner'])
            output['offsets'].append((token['characterOffsetBegin'], token['characterOffsetEnd']))
    return output


def get_str(output, lower=False):
    s = ' '.join(output['word'])
    return s.lower() if lower else s


def normalize_answer(s):
    """Lower text and remove punctuation, storys and extra whitespace."""

    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def find_span(offsets, start, end):
    start_index = end_index = -1
    for i, offset in enumerate(offsets):
        if (start_index < 0) or (start >= offset[0]):
            start_index = i
        if (end_index < 0) and (end <= offset[1]):
            end_index = i
    return (start_index, end_index)


def gen_data(data_file, output_file1, output_file2):
    """
    Preprocess dataset,
    data_file: original train or dev json dataset file,
    output_file1: Json file for the DrQA model,
    output_file2: Files for the seq2seq model.
    """
    with open(data_file, 'r') as f:
        dataset = json.load(f)

    f_src = open('{}-src.txt'.format(output_file2), 'w')
    f_tgt = open('{}-tgt.txt'.format(output_file2), 'w')
    data = []
    start_time = time.time()
    for i, datum in enumerate(dataset['data']):
        if i % 10 == 0:
            print('processing %d / %d (used_time = %.2fs)...' %
                  (i, len(dataset['data']), time.time() - start_time))
        context_str = datum['story']
        _datum = {'context': context_str,
                  'source': datum['source'],
                  'id': datum['id'],
                  'filename': datum['filename']}
        _datum['annotated_context'] = process(context_str)
        _datum['qas'] = []
        _datum['context'] += UNK
        _datum['annotated_context']['word'].append(UNK)
        _datum['annotated_context']['offsets'].append(
            (len(_datum['context']) - len(UNK), len(_datum['context'])))
        assert len(datum['questions']) == len(datum['answers'])

        additional_answers = {}
        if 'additional_answers' in datum:
            for k, answer in datum['additional_answers'].items():
                if len(answer) == len(datum['answers']):
                    for ex in answer:
                        idx = ex['turn_id']
                        if idx not in additional_answers:
                            additional_answers[idx] = []
                        additional_answers[idx].append(ex['input_text'])

        for question, answer in zip(datum['questions'], datum['answers']):
            assert question['turn_id'] == answer['turn_id']
            idx = question['turn_id']
            _qas = {'turn_id': idx,
                    'question': question['input_text'],
                    'answer': answer['input_text']}
            if idx in additional_answers:
                _qas['additional_answers'] = additional_answers[idx]

            _qas['annotated_question'] = process(question['input_text'])
            _qas['annotated_answer'] = process(answer['input_text'])
            _qas['answer_span_start'] = answer['span_start']
            _qas['answer_span_end'] = answer['span_end']

            start = answer['span_start']
            end = answer['span_end']
            chosen_text = _datum['context'][start: end].lower()
            while len(chosen_text) > 0 and chosen_text[0] in string.whitespace:
                chosen_text = chosen_text[1:]
                start += 1
            while len(chosen_text) > 0 and chosen_text[-1] in string.whitespace:
                chosen_text = chosen_text[:-1]
                end -= 1
            input_text = _qas['answer'].strip().lower()
            if input_text in chosen_text:
                i = chosen_text.find(input_text)
                _qas['answer_span'] = find_span(_datum['annotated_context']['offsets'],
                                                start + i, start + i + len(input_text))
            else:
                _qas['answer_span'] = find_span(_datum['annotated_context']['offsets'], start, end)
            _datum['qas'].append(_qas)

            s, e = _qas['answer_span']
            span_str = ' '.join(_datum['annotated_context']['word'][s: e + 1]).lower()
            f_src.write('{} || {}\n'.format(get_str(_qas['annotated_question'], lower=True), span_str))
            f_tgt.write('{}\n'.format(get_str(_qas['annotated_answer'], lower=True)))
        data.append(_datum)
    f_src.close()
    f_tgt.close()

    dataset['data'] = data
    with open(output_file1, 'w') as output_file:
        json.dump(dataset, output_file, sort_keys=True, indent=4)

In [24]:
original_train = "data/coqa-train-v1.0.json" 
orig_train_output1 = "data/coqa.train.pipeline.json"
orig_train_output2 = "data/seq2seq-train-pipeline"
original_dev = "data/coqa-dev-v1.0.json" 
orig_dev_output1 = "data/coqa.dev.pipeline.json"
orig_dev_output2 = "data/seq2seq-dev-pipeline"

In [25]:
gen_data(original_train, orig_train_output1, orig_train_output2)
gen_data(original_dev, orig_dev_output1, orig_dev_output2)

processing 0 / 7199 (used_time = 0.00s)...
processing 10 / 7199 (used_time = 2.81s)...
processing 20 / 7199 (used_time = 5.40s)...
processing 30 / 7199 (used_time = 8.22s)...
processing 40 / 7199 (used_time = 11.29s)...
processing 50 / 7199 (used_time = 13.84s)...
processing 60 / 7199 (used_time = 16.08s)...
processing 70 / 7199 (used_time = 18.86s)...
processing 80 / 7199 (used_time = 20.86s)...
processing 90 / 7199 (used_time = 23.36s)...
processing 100 / 7199 (used_time = 26.48s)...
processing 110 / 7199 (used_time = 28.99s)...
processing 120 / 7199 (used_time = 31.49s)...
processing 130 / 7199 (used_time = 33.51s)...
processing 140 / 7199 (used_time = 36.28s)...
processing 150 / 7199 (used_time = 38.67s)...
processing 160 / 7199 (used_time = 41.28s)...
processing 170 / 7199 (used_time = 44.03s)...
processing 180 / 7199 (used_time = 46.35s)...
processing 190 / 7199 (used_time = 48.76s)...
processing 200 / 7199 (used_time = 51.42s)...
processing 210 / 7199 (used_time = 53.88s)...
pro

### Use NMT process data

```!pip install OpenNMT-py```

In [26]:
!onmt_preprocess -train_src data/seq2seq-train-pipeline-src.txt -train_tgt data/seq2seq-train-pipeline-tgt.txt -valid_src data/seq2seq-dev-pipeline-src.txt -valid_tgt data/seq2seq-dev-pipeline-tgt.txt -save_data data/seq2seq-pipeline -lower -dynamic_dict -src_seq_length 10000

[2020-08-21 02:40:53,866 INFO] Extracting features...
[2020-08-21 02:40:53,866 INFO]  * number of source features: 0.
[2020-08-21 02:40:53,867 INFO]  * number of target features: 0.
[2020-08-21 02:40:53,867 INFO] Building `Fields` object...
[2020-08-21 02:40:53,867 INFO] Building & saving training data...
[2020-08-21 02:40:53,867 WARNING] Shards for corpus train already exist, won't be overwritten, pass the `-overwrite` option if you want to.
[2020-08-21 02:40:53,867 INFO] Building & saving validation data...
[2020-08-21 02:40:53,868 WARNING] Shards for corpus valid already exist, won't be overwritten, pass the `-overwrite` option if you want to.


### Embedding to torch

In [1]:
from __future__ import division
import six
import argparse
import torch
import logging

logger = logging.getLogger()

def _old_style_vocab(vocab):
    """Detect old-style vocabs (``List[Tuple[str, torchtext.data.Vocab]]``).
    Args:
        vocab: some object loaded from a *.vocab.pt file
    Returns:
        Whether ``vocab`` is a list of pairs where the second object
        is a :class:`torchtext.vocab.Vocab` object.
    This exists because previously only the vocab objects from the fields
    were saved directly, not the fields themselves, and the fields needed to
    be reconstructed at training and translation time.
    """

    return isinstance(vocab, list) and \
        any(isinstance(v[1], Vocab) for v in vocab)

def get_vocabs(dict_path):

    fields = torch.load(dict_path)

    vocs = []
    for side in ['src', 'tgt']:
        if _old_style_vocab(fields):
            vocab = next((v for n, v in fields if n == side), None)
        else:
            try:
                vocab = fields[side].base_field.vocab
            except AttributeError:
                vocab = fields[side].vocab
        vocs.append(vocab)
    enc_vocab, dec_vocab = vocs

    logger.info("From: %s" % dict_path)
    logger.info("\t* source vocab: %d words" % len(enc_vocab))
    logger.info("\t* target vocab: %d words" % len(dec_vocab))

    return enc_vocab, dec_vocab


def read_embeddings(file_enc, skip_lines=0, filter_set=None):
    embs = dict()
    total_vectors_in_file = 0
    with open(file_enc, 'rb') as f:
        for i, line in enumerate(f):
            if i < skip_lines:
                continue
            if not line:
                break
            if len(line) == 0:
                # is this reachable?
                continue

            l_split = line.decode('utf8').strip().split(' ')
            if len(l_split) == 2:
                continue
            total_vectors_in_file += 1
            if filter_set is not None and l_split[0] not in filter_set:
                continue
            embs[l_split[0]] = [float(em) for em in l_split[1:]]
    return embs, total_vectors_in_file


def convert_to_torch_tensor(word_to_float_list_dict, vocab):
    dim = len(six.next(six.itervalues(word_to_float_list_dict)))
    tensor = torch.zeros((len(vocab), dim))
    for word, values in word_to_float_list_dict.items():
        tensor[vocab.stoi[word]] = torch.Tensor(values)
    return tensor


def calc_vocab_load_stats(vocab, loaded_embed_dict):
    matching_count = len(
        set(vocab.stoi.keys()) & set(loaded_embed_dict.keys()))
    missing_count = len(vocab) - matching_count
    percent_matching = matching_count / len(vocab) * 100
    return matching_count, missing_count, percent_matching


def embeddings_to_torch(emb_file_enc, emb_file_dec, output_file, dict_file, verbose=False, skip_lines=0, embed_type="Glove"):

    enc_vocab, dec_vocab = get_vocabs(dict_file)
    print("test")
    # Read in embeddings
    logger.info("Reading encoder embeddings from {}".format(
            emb_file_enc))
    src_vectors, total_vec_count = read_embeddings(
            emb_file_enc, skip_lines,
            filter_set=enc_vocab.stoi
        )
    logger.info("\tFound {} total vectors in file.".format(
            total_vec_count))
    logger.info("Reading decoder embeddings from {}".format(
            emb_file_dec))
    tgt_vectors, total_vec_count = read_embeddings(
            emb_file_dec, skip_lines,
            filter_set=dec_vocab.stoi
        )
    logger.info("\tFound {} total vectors in file".format(total_vec_count))
    logger.info("After filtering to vectors in vocab:")
    logger.info("\t* enc: %d match, %d missing, (%.2f%%)"
                % calc_vocab_load_stats(enc_vocab, src_vectors))
    logger.info("\t* dec: %d match, %d missing, (%.2f%%)"
                % calc_vocab_load_stats(dec_vocab, tgt_vectors))

    # Write to file
    enc_output_file = output_file + ".enc.pt"
    dec_output_file = output_file + ".dec.pt"
    logger.info("\nSaving embedding as:\n\t* enc: %s\n\t* dec: %s"
                % (enc_output_file, dec_output_file))
    torch.save(
        convert_to_torch_tensor(src_vectors, enc_vocab),
        enc_output_file
    )
    torch.save(
        convert_to_torch_tensor(tgt_vectors, dec_vocab),
        dec_output_file
    )
    logger.info("\nDone.")


In [2]:
emb_file_enc = "wordvecs/glove.42B.300d.txt" 
emb_file_dec = "wordvecs/glove.42B.300d.txt" 
dict_file    = "data/seq2seq-pipeline.vocab.pt"
output_file  = "data/seq2seq-pipeline.embed"

embeddings_to_torch(emb_file_enc, emb_file_dec, output_file, dict_file)

test


# 4. Train the model
### 4.1 First use dataset to train DrQA Model

In [ ]:
# Running on CPU
!python main.py --trainset data/coqa.train.pipeline.json --devset data/coqa.dev.pipeline.json --n_history 0 --dir pipeline_models --embed_file wordvecs/glove.840B.300d.txt --predict_raw_text n --cuda False

#Run network on cuda
# !python main.py --trainset data/coqa.train.pipeline.json --devset data/coqa.dev.pipeline.json --n_history 0 --dir pipeline_models --embed_file wordvecs/glove.840B.300d.txt --predict_raw_text n --cuda True

**************** MODEL CONFIGURATION ****************
batch_size               -->   32
cased                    -->   True
concat_rnn_layers        -->   True
cuda                     -->   True
cuda_id                  -->   -1
debug                    -->   False
devset                   -->   data/coqa.dev.pipeline.json
dir                      -->   pipeline_models
doc_self_attn            -->   False
dropout_emb              -->   0.5
dropout_ff               -->   0.5
dropout_rnn              -->   0.3
dropout_rnn_output       -->   True
embed_file               -->   wordvecs/glove.840B.300d.txt
embed_size               -->   None
embed_type               -->   glove
f_ner                    -->   False
f_pos                    -->   False
f_qem                    -->   True
fix_embeddings           -->   False
grad_clipping            -->   10.0
hidden_size              -->   300
learning_rate            -->   0.1
max_answer_len           -->   15
max_epochs               --> 

### 4.2 Second use the evidence generated from DrQA model to train PGNet model

In [3]:
!onmt_train -data data/seq2seq-pipeline -save_model pipeline_models/seq2seq_copy -copy_attn -reuse_copy_attn -word_vec_size 300 -pre_word_vecs_enc data/seq2seq-pipeline.embed.enc.pt -pre_word_vecs_dec data/seq2seq-pipeline.embed.dec.pt -train_steps 50

[2020-08-21 20:08:39,910 INFO]  * src vocab size = 47388
[2020-08-21 20:08:39,910 INFO]  * tgt vocab size = 33978
[2020-08-21 20:08:39,910 INFO] Building model...
[2020-08-21 20:08:42,655 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(47388, 300, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(300, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(33978, 300, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False)
      (layers): ModuleList(
        (0): LSTMCell(800, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
      

In [ ]:
!python main.py --testset data/coqa.dev.pipeline.json --n_history 2 --pretrained pipeline_models

In [6]:
import json
def gen_seq2seq(data_file, output_file, pred_file):
    with open(data_file) as f:
        dataset = json.load(f)

    questions = {}
    for i, datum in enumerate(dataset['data']):
        for qas in datum['qas']:
            idx = datum['id'] + ':' + str(qas['turn_id'])
            questions[idx] = ' '.join(qas['annotated_question']['word'])

    with open(pred_file) as f:
        predictions = json.load(f)

    f_out = open(output_file, 'w')
    for prediction in predictions:
        idx = prediction['id'] + ':' + str(prediction['turn_id'])
        f_out.write(questions[idx].lower() + ' || ' + prediction['answer'].replace('\n', ' ').lower() + '\n')
    f_out.close()


def gen_output(data_file, pred_file, output_file):
    with open(data_file) as f:
        dataset = json.load(f)

    output = []
    for i, datum in enumerate(dataset['data']):
        print('processing {}/{}...'.format(i, len(dataset['data'])))
        for question, answer in zip(datum['questions'], datum['answers']):
            assert question['turn_id'] == answer['turn_id']
            output.append({'id': datum['id'], 'turn_id': question['turn_id']})

    predictions = []
    with open(pred_file) as f:
        for line in f.readlines():
            predictions.append(line.strip())

    assert len(output) == len(predictions)
    for out, pred in zip(output, predictions):
        out['answer'] = pred

    with open(output_file, 'w') as outfile:
        json.dump(output, outfile, indent=4)

In [9]:
data_file = "data/coqa.dev.pipeline.json"
output_file = "pipeline_models/pipeline-seq2seq-src.txt" 
pred_file = "pipeline_models/predictions.json"
gen_seq2seq(data_file, output_file, pred_file)

In [20]:
!onmt_translate -model pipeline_models/seq2seq_copy_step_5.pt -src data/seq2seq-dev-pipeline-src.txt -output pipeline_models/pred.txt -replace_unk -verbose

[2020-08-20 11:04:00,340 INFO] Translating shard 0.
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)
[2020-08-20 11:04:03,507 INFO] 
SENT 1: ['what', 'color', 'was', 'cotton', '?', '||', 'white']
PRED 1: 
PRED SCORE: -0.0000

[2020-08-20 11:04:03,521 INFO] 
SENT 2: ['where', 'did', 'she', 'live', '?', '||', 'in', 'a', 'barn']
PRED 2: 
PRED SCORE: -0.0000

[2020-08-20 11:04:03,522 INFO] 
SENT 3: ['did', 'she', 'live', 'alone', '?', '||', 'cotton', 'was', "n't", 'alone']
PRED 3: 
PRED SCORE: -0.0000

[2020-08-20 11:04:03,522 INFO] 
SENT 4: ['who', 'did', 'she', 'live', 'with', '?', '||', 'with', 'her', 'mommy', 'and', '5', 'other', 'sisters']
PRED 4: 
PRED SCORE: -0.0000

[2020-08-20 11:04:03,522 

In [ ]:
data_file = "data/coqa-dev-v1.0.json"
pred_file = "pipeline_models/pred.txt"
output_file = "pipeline_models/pipeline.prediction.json"
gen_output(data_file, pred_file, output_file)

# Test 
Reduce training process steps to save time.

In [2]:
!python main.py --trainset data/coqa.train.pipeline.json --devset data/coqa.dev.pipeline.json --n_history 0 --dir pipeline_models --embed_file wordvecs/glove.840B.300d.txt --predict_raw_text n

**************** MODEL CONFIGURATION ****************
batch_size               -->   32
cased                    -->   True
concat_rnn_layers        -->   True
cuda                     -->   True
cuda_id                  -->   -1
debug                    -->   False
devset                   -->   data/coqa.dev.pipeline.json
dir                      -->   pipeline_models
doc_self_attn            -->   False
dropout_emb              -->   0.5
dropout_ff               -->   0.5
dropout_rnn              -->   0.3
dropout_rnn_output       -->   True
embed_file               -->   wordvecs/glove.840B.300d.txt
embed_size               -->   None
embed_type               -->   glove
f_ner                    -->   False
f_pos                    -->   False
f_qem                    -->   True
fix_embeddings           -->   False
grad_clipping            -->   10.0
hidden_size              -->   300
learning_rate            -->   0.1
max_answer_len           -->   15
max_epochs               --> 

In [8]:
!python main.py --testset data/coqa.dev.pipeline.json --n_history 2 --pretrained pipeline_models

**************** MODEL CONFIGURATION ****************
batch_size               -->   32
cased                    -->   True
concat_rnn_layers        -->   True
cuda                     -->   True
cuda_id                  -->   -1
debug                    -->   False
devset                   -->   None
dir                      -->   None
doc_self_attn            -->   False
dropout_emb              -->   0.5
dropout_ff               -->   0.5
dropout_rnn              -->   0.3
dropout_rnn_output       -->   True
embed_file               -->   None
embed_size               -->   None
embed_type               -->   glove
f_ner                    -->   False
f_pos                    -->   False
f_qem                    -->   True
fix_embeddings           -->   False
grad_clipping            -->   10.0
hidden_size              -->   300
learning_rate            -->   0.1
max_answer_len           -->   15
max_epochs               -->   50
min_freq                 -->   20
momentum           

In [ ]:
data_file = "data/coqa.dev.pipeline.json"
output_file = "pipeline_models/pipeline-seq2seq-src.txt" 
pred_file = "pipeline_models/predictions.json"
gen_seq2seq(data_file, output_file, pred_file)

In [ ]:
!onmt_translate -model pipeline_models/seq2seq_copy_step_50.pt -src data/seq2seq-dev-pipeline-src.txt -output pipeline_models/pred.txt -replace_unk -verbose

[2020-08-21 20:30:35,168 INFO] Translating shard 0.
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)
[2020-08-21 20:32:11,882 INFO] 
SENT 1: ['what', 'color', 'was', 'cotton', '?', '||', 'white']
PRED 1: white white white white ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
PRED SCORE: -111.7016

[2020-08-21 20:32:11,883 INFO] 
SENT 2: ['where', 'did', 'she', 'live', '?', '||', 'in', 'a', 'barn']
PRED 2: in in a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a

In [ ]:
data_file = "data/coqa-dev-v1.0.json"
pred_file = "pipeline_models/pred.txt"
output_file = "pipeline_models/pipeline.prediction.json"
gen_output(data_file, pred_file, output_file)